## Book Recommender System using GoodReads Ratings

This Jupyter notebook implements a book recommender system using ratings data from Goodreads, a popular online platform for book enthusiasts.

In [19]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import data

from IPython.display import Markdown

In [20]:
# configurable parameters, change as needed

# set to true if loading existing model file, false if training a new model
skip_training = True
explicit = False
data_dir = 'data'
model_save_path = 'models/goodreads_recsys.pth'

pd.options.display.max_rows = None

In [21]:
# create dirs if not existing
os.makedirs(data_dir, exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)

In [22]:
# additional settings, automatically selects cuda if available
if skip_training:
    device_type = 'cpu'
elif torch.cuda.is_available():
    device_type = 'cuda:0'
else:
    device_type = 'cpu'

# set manually if needed e.g. device_type = 'cpu'
print("Using device type:", device_type)
device = torch.device(device_type)

Using device type: cpu


In [23]:
trainset = data.GoodReadsRatingsDataset(root=data_dir, mode='train', explicit=explicit)
testset = data.GoodReadsRatingsDataset(root=data_dir, mode='test', explicit=explicit)

Dataset: train, User count: 6769, Book count: 92034, Ratings count: 966946
Dataset: test, User count: 6376, Book count: 79997, Ratings count: 288004


In [24]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1024, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

In [25]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        super(RecommenderSystem, self).__init__()

        self.user_em = nn.Embedding(n_users, 100)
        self.item_em = nn.Embedding(n_items, 100)
        self.drop0 = nn.Dropout(0.02)
        
        self.fc1 = nn.Linear(200, 100)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.02)
        
        self.fc2 = nn.Linear(100, 10)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.02)
        
        self.fc3 = nn.Linear(10, 1)
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 0).
          item_ids of shape (batch_size): Item ids (starting from 0).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        x = torch.cat([self.user_em(user_ids), self.item_em(item_ids)], dim=1)
        x = self.drop0(x)

        x = self.fc1(x)
        x = self.relu1(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.drop2(x)

        x = torch.sigmoid(self.fc3(x))
        # min_rating, max_rating = (0.5, 5.5)
        # x = x*(max_rating - min_rating) + min_rating
        x = x.view(-1)
        
        return x

In [26]:
model = RecommenderSystem(trainset.n_users, trainset.n_items)
model.to(device)

RecommenderSystem(
  (user_em): Embedding(6769, 20)
  (item_em): Embedding(92034, 100)
  (drop0): Dropout(p=0.02, inplace=False)
  (fc1): Linear(in_features=120, out_features=100, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.02, inplace=False)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.02, inplace=False)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
)

In [27]:
# computes the loss:
def compute_loss(model, testloader):
    model.eval()
    cost = nn.MSELoss()
    total_loss = 0
    total_data = 0
    prediction_list = []

    with torch.no_grad():
        for user_ids, item_ids, labels in testloader:
            user_ids, item_ids, labels = user_ids.to(device), item_ids.to(device), labels.to(device)
            predictions = model(user_ids, item_ids)
            prediction_list.append(predictions.cpu())

            loss = cost(predictions, labels)
            total_loss += (loss.item() * labels.size(0))
            total_data += labels.size(0)

    loss = total_loss / total_data
    return torch.cat(prediction_list), loss

In [28]:
# training loop
if not skip_training:
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
    cost = nn.MSELoss()
    total_loss = 0
    total_data = 0

    # save historical losses and accs
    hist_metrics = dict()
    hist_metrics['epoch'] = []
    hist_metrics['train_loss'] = []
    hist_metrics['test_loss'] = []

    epochs = 20
    for epoch in range(epochs):
        model.train()
        for user_ids, item_ids, labels in trainloader:  
            user_ids, item_ids, labels = user_ids.to(device), item_ids.to(device), labels.to(device)

            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)

            loss = cost(predictions, labels)
            total_loss += (loss.item() * labels.size(0))
            total_data += labels.size(0)
            loss.backward()
            optimizer.step()

        train_loss = total_loss / total_data
        _, test_loss = compute_loss(model, testloader)

        hist_metrics['epoch'].append(epoch)
        hist_metrics['train_loss'].append(train_loss)
        hist_metrics['test_loss'].append(test_loss)

        print('Epoch {}: Train error: {:.4f}, Test error: {:.4f}'.format(epoch, train_loss, test_loss))

In [29]:
# plot and save historical train/test loss
def plot_metrics(metrics, save_path='logs/goodreads_recsys_{}.{}'):
    plt.plot(metrics['train_loss'])
    plt.plot(metrics['test_loss'])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(save_path.format('loss', "png"))
    plt.show()

    with open(save_path.format("hist", "json"), 'w') as f:
        json.dump(metrics, f)

In [30]:
# save trained model
if not skip_training:
    torch.save(model.state_dict(), model_save_path)
    plot_metrics(hist_metrics)

In [31]:
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    model.load_state_dict(torch.load(model_save_path, map_location=lambda storage, loc: storage))
    print('Model loaded from: {}'.format(model_save_path))
    model.to(device)
    model.eval()

Model loaded from: models/goodreads_recsys.pth


In [32]:
predictions, loss = compute_loss(model, testloader)
print('Test loss: {:.4f}'.format(loss))

Test loss: 0.1810


In [33]:
book_titles_df = pd.read_csv(os.path.join(data_dir, 'book_titles.csv'), header=0)
book_titles_df.set_index('encoded_book_id', inplace=True)

In [34]:
next_user_i = 0
top_half_hit = 0
top10_hit = 0
top01_hit = 0
count = 0

while next_user_i < len(testset):
    user_id = testset[next_user_i][0]
    last_user_i = next_user_i
    while next_user_i < len(testset) and testset[next_user_i][0] == user_id:
        next_user_i += 1
    indices = range(last_user_i, next_user_i)

    labels = testset[indices][2]
    _, top_reco = torch.topk(predictions[indices], k=int(len(indices)/2))
    top_half_hit += labels[top_reco].sum()/int(len(indices)/2)

    _, top_reco = torch.topk(predictions[indices], k=min(int(len(indices)/2), 10))
    top10_hit += labels[top_reco].sum()/min(int(len(indices)/2), 10)

    _, top_reco = torch.topk(predictions[indices], k=1)
    top01_hit += labels[top_reco].sum()

    count += 1


top_half_hit = top_half_hit / count
print("top 50% hit rate:", top_half_hit.item())
top10_hit = top10_hit / count
print("top 10 hit rate:", top10_hit.item())
top01_hit = top01_hit / count
print("top 1 hit rate:", top01_hit.item())

top 50% hit rate: 0.7229639291763306
top 10 hit rate: 0.7837969064712524
top 1 hit rate: 0.8602572083473206


In [46]:
indices = testset.get_user_record(random=False, user_id=5)
sampleset = torch.utils.data.Subset(testset, indices)
sampleloader = torch.utils.data.DataLoader(sampleset, batch_size=len(indices), shuffle=False)

if explicit:
    max_range = 5.0
else:
    max_range = 1.0

with torch.no_grad():
    for user_ids, item_ids, labels in sampleloader:
        print("Showing (sample) books from user_enc_id={}".format(user_ids[0]))
        user_ids, item_ids, labels = user_ids.to(device), item_ids.to(device), labels.to(device)
        predictions = model(user_ids, item_ids)

        tmp_df = book_titles_df.loc[item_ids.cpu()]
        tmp_df['actual_ratings'] = labels.cpu() * max_range
        tmp_df['predicted_ratings'] = predictions.cpu().round(decimals=2) * max_range
        tmp_df['correct'] = (tmp_df['predicted_ratings'].round() == tmp_df['actual_ratings'])
        
        # display title of correct predictions as green
        # tmp_df.loc[tmp_df['correct'], 'title'] = '<span style="color: #00ff00">' + tmp_df.loc[tmp_df['correct'], 'title'] + '</span>'
        # display title of correct predictions as red
        # tmp_df.loc[~tmp_df['correct'], 'title'] = '<span style="color: #ff0000">' + tmp_df.loc[~tmp_df['correct'], 'title'] + '</span>'
        
        tmp_df['read?'] = '<span style="color: #ff0000">no</span>'
        tmp_df.loc[tmp_df['actual_ratings'] == 1, 'read?'] = '<span style="color: #00ff00">yes</span>'
        
        tmp_df['recommend?'] = '<span style="color: #ff0000">no</span>'
        tmp_df.loc[tmp_df['predicted_ratings'] >= 0.5, 'recommend?'] = '<span style="color: #00ff00">yes</span>'

        tmp_df['correctly recommended?'] = '<span style="color: #ff0000">no</span>'
        tmp_df.loc[tmp_df['correct'] == 1, 'correctly recommended?'] = '<span style="color: #00ff00">yes</span>'

        print("{:.2f}% of the books have been correctly recommended".format(tmp_df['correct'].sum() / len(tmp_df['correct']) * 100))
        print("{:.2f}% of the books from read have been correctly recommended".format(tmp_df.loc[tmp_df['actual_ratings'] == 1, 'correct'].sum() / tmp_df['actual_ratings'].sum() * 100))

        tmp_df = tmp_df.sample(n=min(len(tmp_df.index), 40))
        tmp_df.sort_values(by=['actual_ratings', 'title'], ascending=[False, True], inplace=True)
        display(Markdown(tmp_df[['book_id', 'title', 'read?', 'recommend?', 'correctly recommended?']].to_markdown()))

Showing (sample) books from user_enc_id=5
67.95% of the books have been correctly recommended
45.26% of the books from read have been correctly recommended


|   encoded_book_id |   book_id | title                                                        | read?                                   | recommend?                              | correctly recommended?                  |
|------------------:|----------:|:-------------------------------------------------------------|:----------------------------------------|:----------------------------------------|:----------------------------------------|
|             82742 |  23766634 | A Court of Wings and Ruin (A Court of Thorns and Roses, #3)  | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             76497 |  22055262 | A Darker Shade of Magic (Shades of Magic, #1)                | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             72746 |  20560137 | An Ember in the Ashes (An Ember in the Ashes, #1)            | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             86775 |  25614934 | Arcadia                                                      | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             29927 |   7326784 | Black Bird, Vol. 05 (Black Bird, #5)                         | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             44130 |  12803252 | Black Bird, Vol. 14 (Black Bird, #14)                        | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             87560 |  25852870 | Eligible (The Austen Project, #4)                            | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             77730 |  22489107 | Fairest (The Lunar Chronicles, #3.5)                         | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|              2870 |     68488 | Fool's Errand (Tawny Man, #1)                                | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             75491 |  21851568 | Hidden Huntress (The Malediction Trilogy, #2)                | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             64489 |  18089930 | Innocent Blood (The Order of the Sanguines, #2)              | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             52538 |  15790883 | Promise of Blood (Powder Mage, #1)                           | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             59308 |  17331819 | Reawakened (Reawakened, #1)                                  | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             90459 |  28818314 | RoseBlood                                                    | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             35335 |   9466068 | Silk Is for Seduction (The Dressmakers, #1)                  | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             81484 |  23437156 | Six of Crows (Six of Crows, #1)                              | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             64710 |  18114110 | Sword of the North (Grim Company, #2)                        | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             30144 |   7455727 | Tails of Wonder and Imagination                              | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             53878 |  15985344 | The City                                                     | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             63787 |  18004320 | The Last Ever After (The School for Good and Evil, #3)       | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             35520 |   9532302 | The Last Werewolf (The Last Werewolf, #1)                    | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             34131 |   8709528 | The Ruby Circle (Bloodlines, #6)                             | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> | <span style="color: #00ff00">yes</span> |
|             81535 |  23444482 | The Traitor Baru Cormorant (Baru Cormorant #1)               | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             38016 |  10425771 | The Twelfth Enchantment                                      | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             83878 |  24585267 | The White Rose (The Lone City, #2)                           | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|              4544 |    138473 | Tsubasa: RESERVoir CHRoNiCLE, Vol. 15                        | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  |
|             47718 |  13500978 | Call Me Wild (Wild Thing #2)                                 | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|              7580 |    323253 | Gabe (Buckhorn Brothers, #3)                                 | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             24401 |   5941383 | Girlfriend Material                                          | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             16336 |   1214613 | Listen to Your Heart                                         | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             74789 |  21461416 | My Little Soldiers                                           | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             45936 |  13182417 | Sakura Hime: The Legend of Princess Sakura, Vol. 9           | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             44676 |  12958873 | Sparks (Something in Common, #2)                             | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             21473 |   2919954 | Swan Song                                                    | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|              8292 |    370897 | The Gods of Riverworld (Riverworld, #5)                      | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             60766 |  17571908 | The Redhead Plays Her Hand (Redhead, #3)                     | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> | <span style="color: #ff0000">no</span>  |
|             13312 |    811191 | The Secret Life of Sparrow Delaney                           | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             26528 |   6436047 | Tooth and Claw                                               | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|              4018 |    114123 | Valley of Silence (Circle trilogy #3)                        | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |
|             87711 |  25910177 | You Only Live Nine Times (Supernatural Enforcers Agency, #3) | <span style="color: #ff0000">no</span>  | <span style="color: #ff0000">no</span>  | <span style="color: #00ff00">yes</span> |

In [55]:
tmp_df = None
# indices = testset.get_user_record()
print(testset[indices[0]][0])
user_ids = testset[indices[0]][0].repeat(book_titles_df.shape[0])
item_ids = torch.LongTensor(book_titles_df.index.values)

with torch.no_grad():
    user_ids, item_ids = user_ids.to(device), item_ids.to(device)
    predictions = model(user_ids, item_ids)

    # tmp_df = pd.DataFrame(index=item_ids.numpy(), columns=['book_id', 'title', 'predicted_ratings'])
    tmp_df = book_titles_df.copy()
    tmp_df['predicted_ratings'] = predictions.cpu().round(decimals=2) * max_range
    tmp_df.sort_values(by=['predicted_ratings'], ascending=False, inplace=True)
    tmp_df = tmp_df.loc[tmp_df['title'].notna()]
    tmp_df['read?'] = '<span style="color: #ff0000">no</span>'
    

    for user_ids, item_ids, labels in sampleloader:
        tmp_df[labels.nonzero().squeeze(dim=1), 'read?'] = '<span style="color: #00ff00">yes</span>'

    display(Markdown(tmp_df[['title', 'predicted_ratings', 'read?']].head(100).to_markdown()))

tensor(5)


|   encoded_book_id | title                                                                       |   predicted_ratings | read?                                  |
|------------------:|:----------------------------------------------------------------------------|--------------------:|:---------------------------------------|
|                 2 | Harry Potter and the Sorcerer's Stone (Harry Potter, #1)                    |                0.95 | <span style="color: #ff0000">no</span> |
|                 4 | Harry Potter and the Goblet of Fire (Harry Potter, #4)                      |                0.95 | <span style="color: #ff0000">no</span> |
|               728 | Harry Potter and the Chamber of Secrets (Harry Potter, #2)                  |                0.95 | <span style="color: #ff0000">no</span> |
|                 0 | Harry Potter and the Half-Blood Prince (Harry Potter, #6)                   |                0.94 | <span style="color: #ff0000">no</span> |
|              4516 | Harry Potter and the Deathly Hallows (Harry Potter, #7)                     |                0.94 | <span style="color: #ff0000">no</span> |
|                18 | The Fellowship of the Ring (The Lord of the Rings, #1)                      |                0.93 | <span style="color: #ff0000">no</span> |
|                 1 | Harry Potter and the Order of the Phoenix (Harry Potter, #5)                |                0.93 | <span style="color: #ff0000">no</span> |
|                57 | Pride and Prejudice                                                         |                0.93 | <span style="color: #ff0000">no</span> |
|               438 | A Game of Thrones (A Song of Ice and Fire, #1)                              |                0.93 | <span style="color: #ff0000">no</span> |
|              1165 | The Lightning Thief (Percy Jackson and the Olympians, #1)                   |                0.92 | <span style="color: #ff0000">no</span> |
|              5298 | The Name of the Wind (The Kingkiller Chronicle, #1)                         |                0.92 | <span style="color: #ff0000">no</span> |
|             24562 | Blood Promise (Vampire Academy, #4)                                         |                0.92 | <span style="color: #ff0000">no</span> |
|               160 | The Hobbit                                                                  |                0.92 | <span style="color: #ff0000">no</span> |
|               634 | The Two Towers (The Lord of the Rings, #2)                                  |                0.92 | <span style="color: #ff0000">no</span> |
|               602 | Neverwhere                                                                  |                0.92 | <span style="color: #ff0000">no</span> |
|               843 | The Return of the King (The Lord of the Rings, #3)                          |                0.92 | <span style="color: #ff0000">no</span> |
|                 8 | The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1) |                0.92 | <span style="color: #ff0000">no</span> |
|              3697 | The Lion, the Witch, and the Wardrobe (Chronicles of Narnia, #1)            |                0.92 | <span style="color: #ff0000">no</span> |
|              1943 | Lover Revealed (Black Dagger Brotherhood, #4)                               |                0.91 | <span style="color: #ff0000">no</span> |
|              1164 | The Sea of Monsters (Percy Jackson and the Olympians, #2)                   |                0.91 | <span style="color: #ff0000">no</span> |
|             23372 | The Last Olympian (Percy Jackson and the Olympians, #5)                     |                0.91 | <span style="color: #ff0000">no</span> |
|             23798 | Lover Avenged (Black Dagger Brotherhood, #7)                                |                0.91 | <span style="color: #ff0000">no</span> |
|              2857 | The Well of Ascension (Mistborn, #2)                                        |                0.91 | <span style="color: #ff0000">no</span> |
|             24232 | Along for the Ride                                                          |                0.91 | <span style="color: #ff0000">no</span> |
|             46612 | Opal (Lux, #3)                                                              |                0.91 | <span style="color: #ff0000">no</span> |
|              4562 | Dead to the World (Sookie Stackhouse, #4)                                   |                0.91 | <span style="color: #ff0000">no</span> |
|              2856 | The Final Empire (Mistborn, #1)                                             |                0.91 | <span style="color: #ff0000">no</span> |
|             29535 | Clockwork Angel (The Infernal Devices, #1)                                  |                0.91 | <span style="color: #ff0000">no</span> |
|              1944 | Dark Lover (Black Dagger Brotherhood, #1)                                   |                0.91 | <span style="color: #ff0000">no</span> |
|              2668 | A Storm of Swords (A Song of Ice and Fire, #3)                              |                0.91 | <span style="color: #ff0000">no</span> |
|             38583 | A Dance with Dragons (A Song of Ice and Fire, #5)                           |                0.91 | <span style="color: #ff0000">no</span> |
|              3945 | Living Dead in Dallas (Sookie Stackhouse, #2)                               |                0.91 | <span style="color: #ff0000">no</span> |
|              2031 | Mansfield Park                                                              |                0.91 | <span style="color: #ff0000">no</span> |
|              1700 | Lover Eternal (Black Dagger Brotherhood, #2)                                |                0.91 | <span style="color: #ff0000">no</span> |
|              1960 | Interview with the Vampire (The Vampire Chronicles, #1)                     |                0.91 | <span style="color: #ff0000">no</span> |
|              4267 | Prince Caspian (Chronicles of Narnia, #2)                                   |                0.91 | <span style="color: #ff0000">no</span> |
|               933 | The Princess Bride                                                          |                0.91 | <span style="color: #ff0000">no</span> |
|             26772 | Spirit Bound (Vampire Academy, #5)                                          |                0.91 | <span style="color: #ff0000">no</span> |
|              1945 | Lover Awakened (Black Dagger Brotherhood, #3)                               |                0.91 | <span style="color: #ff0000">no</span> |
|              4236 | The Golden Compass (His Dark Materials, #1)                                 |                0.91 | <span style="color: #ff0000">no</span> |
|             63844 | The Martian                                                                 |                0.91 | <span style="color: #ff0000">no</span> |
|             50867 | Me Before You (Me Before You, #1)                                           |                0.91 | <span style="color: #ff0000">no</span> |
|             19910 | The Graveyard Book                                                          |                0.91 | <span style="color: #ff0000">no</span> |
|              6406 | City of Bones (The Mortal Instruments, #1)                                  |                0.91 | <span style="color: #ff0000">no</span> |
|             51277 | Hopeless (Hopeless, #1)                                                     |                0.91 | <span style="color: #ff0000">no</span> |
|                17 | The Lord of the Rings (The Lord of the Rings, #1-3)                         |                0.91 | <span style="color: #ff0000">no</span> |
|             16340 | The Wise Man's Fear (The Kingkiller Chronicle, #2)                          |                0.91 | <span style="color: #ff0000">no</span> |
|               439 | A Feast for Crows (A Song of Ice and Fire, #4)                              |                0.91 | <span style="color: #ff0000">no</span> |
|               774 | Stardust                                                                    |                0.91 | <span style="color: #ff0000">no</span> |
|             19653 | The Battle of the Labyrinth (Percy Jackson and the Olympians, #4)           |                0.91 | <span style="color: #ff0000">no</span> |
|             18032 | City of Ashes (The Mortal Instruments, #2)                                  |                0.91 | <span style="color: #ff0000">no</span> |
|              7326 | Lover Unbound (Black Dagger Brotherhood, #5)                                |                0.91 | <span style="color: #ff0000">no</span> |
|             20111 | Frostbite (Vampire Academy, #2)                                             |                0.91 | <span style="color: #ff0000">no</span> |
|               790 | Dracula                                                                     |                0.91 | <span style="color: #ff0000">no</span> |
|               827 | A Wrinkle in Time (A Wrinkle in Time Quintet, #1)                           |                0.91 | <span style="color: #ff0000">no</span> |
|               343 | A Clash of Kings  (A Song of Ice and Fire, #2)                              |                0.91 | <span style="color: #ff0000">no</span> |
|             21253 | Shadow Kiss (Vampire Academy, #3)                                           |                0.91 | <span style="color: #ff0000">no</span> |
|             10650 | The Titan's Curse (Percy Jackson and the Olympians, #3)                     |                0.91 | <span style="color: #ff0000">no</span> |
|               357 | The Chronicles of Narnia (Chronicles of Narnia, #1-7)                       |                0.9  | <span style="color: #ff0000">no</span> |
|              2324 | The Truth About Forever                                                     |                0.9  | <span style="color: #ff0000">no</span> |
|              2325 | Just Listen                                                                 |                0.9  | <span style="color: #ff0000">no</span> |
|               421 | Alice in Wonderland                                                         |                0.9  | <span style="color: #ff0000">no</span> |
|             55697 | The Coincidence of Callie & Kayden (The Coincidence, #1)                    |                0.9  | <span style="color: #ff0000">no</span> |
|               385 | 'Salem's Lot                                                                |                0.9  | <span style="color: #ff0000">no</span> |
|             48417 | Easy (Contours of the Heart, #1)                                            |                0.9  | <span style="color: #ff0000">no</span> |
|             37005 | Clockwork Prince (The Infernal Devices, #2)                                 |                0.9  | <span style="color: #ff0000">no</span> |
|               116 | American Gods (American Gods, #1)                                           |                0.9  | <span style="color: #ff0000">no</span> |
|              2722 | Guards! Guards! (Discworld, #8)                                             |                0.9  | <span style="color: #ff0000">no</span> |
|              2061 | Eldest (The Inheritance Cycle, #2)                                          |                0.9  | <span style="color: #ff0000">no</span> |
|              2765 | The Magician's Nephew (Chronicles of Narnia, #6)                            |                0.9  | <span style="color: #ff0000">no</span> |
|              2772 | The Silver Chair (Chronicles of Narnia, #4)                                 |                0.9  | <span style="color: #ff0000">no</span> |
|               104 | A Great and Terrible Beauty (Gemma Doyle, #1)                               |                0.9  | <span style="color: #ff0000">no</span> |
|             40064 | Cinder (The Lunar Chronicles, #1)                                           |                0.9  | <span style="color: #ff0000">no</span> |
|             42360 | The Mark of Athena (The Heroes of Olympus, #3)                              |                0.9  | <span style="color: #ff0000">no</span> |
|             52434 | The Ocean at the End of the Lane                                            |                0.9  | <span style="color: #ff0000">no</span> |
|              1750 | The Tale of Despereaux                                                      |                0.9  | <span style="color: #ff0000">no</span> |
|               828 | Romeo and Juliet                                                            |                0.9  | <span style="color: #ff0000">no</span> |
|              3261 | Dance with the Devil (Dark-Hunter #3)                                       |                0.9  | <span style="color: #ff0000">no</span> |
|             46665 | Slammed (Slammed, #1)                                                       |                0.9  | <span style="color: #ff0000">no</span> |
|               188 | Emma                                                                        |                0.9  | <span style="color: #ff0000">no</span> |
|             45747 | Because of Low (Sea Breeze, #2)                                             |                0.9  | <span style="color: #ff0000">no</span> |
|              1620 | The Color of Magic (Discworld, #1; Rincewind #1)                            |                0.9  | <span style="color: #ff0000">no</span> |
|             43572 | Obsidian (Lux, #1)                                                          |                0.9  | <span style="color: #ff0000">no</span> |
|             42110 | I've Got Your Number                                                        |                0.9  | <span style="color: #ff0000">no</span> |
|              1935 | Something Blue (Darcy & Rachel, #2)                                         |                0.9  | <span style="color: #ff0000">no</span> |
|               536 | The Fires of Heaven (Wheel of Time, #5)                                     |                0.9  | <span style="color: #ff0000">no</span> |
|               517 | Alanna: The First Adventure (Song of the Lioness, #1)                       |                0.9  | <span style="color: #ff0000">no</span> |
|             47645 | Point of Retreat (Slammed, #2)                                              |                0.9  | <span style="color: #ff0000">no</span> |
|               202 | Do Androids Dream of Electric Sheep?                                        |                0.9  | <span style="color: #ff0000">no</span> |
|             52024 | On Dublin Street (On Dublin Street, #1)                                     |                0.9  | <span style="color: #ff0000">no</span> |
|             33338 | Lover Unleashed (Black Dagger Brotherhood, #9)                              |                0.9  | <span style="color: #ff0000">no</span> |
|               316 | The Shadow Rising (Wheel of Time, #4)                                       |                0.9  | <span style="color: #ff0000">no</span> |
|                54 | A Midsummer Night's Dream                                                   |                0.9  | <span style="color: #ff0000">no</span> |
|                37 | The Phantom Tollbooth                                                       |                0.9  | <span style="color: #ff0000">no</span> |
|             22961 | City of Glass (The Mortal Instruments, #3)                                  |                0.9  | <span style="color: #ff0000">no</span> |
|                72 | Anansi Boys                                                                 |                0.9  | <span style="color: #ff0000">no</span> |
|             28469 | This Side of the Grave (Night Huntress, #5)                                 |                0.9  | <span style="color: #ff0000">no</span> |
|             28165 | City of Fallen Angels (The Mortal Instruments, #4)                          |                0.9  | <span style="color: #ff0000">no</span> |
|             29096 | Lover Mine (Black Dagger Brotherhood, #8)                                   |                0.9  | <span style="color: #ff0000">no</span> |
|                68 | Persuasion                                                                  |                0.9  | <span style="color: #ff0000">no</span> |